## More Complex Neural Network Problem

<font size = 3>In order to test out using a more complex neural network, we needed a more complicated problem. This involved building a neural network that could classify AGN with jets into three groups dependent on the bending angle between the jets.

Group 1: ang < 20 degrees (wide tail radio galaxies)

<br>
Group 2: 20 < ang < 45 degrees

Group 3: ang > 45 degrees (bent tail radio galaxies)

Garon et al published a paper in 2019 that measured the bending angle for our previous training set of jet sources, so we could simply use this angle information along with our previous training set for this problem. 

It should be noted that we first attemped to solve this problem using our simple neural network, and this achieved an accuracy of around 90%. 

In [2]:
import tensorflow as tf
import numpy as np
import os
from astropy.io import fits 
from astropy import stats
from scipy.ndimage import rotate 
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
def get_clipped_images(filepath,xpix,ypix,sigma):
    '''
    Put FITS data from desired folder into a 3D array
    sigma = how many sigmas from the median background value to sigma clip the data to
    
    TODO: Need to check for .fits ending so works if other files in directory
    '''
    newpath = filepath.replace(os.sep, '/')
    dirs = os.listdir(newpath)
    n = len(dirs)
    data = np.empty(shape=(n,xpix,ypix),dtype=np.float64)
    for i in range(n):
        fullpath = '{}/{}'.format(newpath,dirs[i])
        d = fits.getdata(fullpath, ext=0)
        d[np.isnan(d)] = 0
        _,median,std = stats.sigma_clipped_stats(d, sigma=sigma)
        d[d<median+sigma*std] = median+sigma*std
        data[i,:,:] = d
    return data

def augment_data(data,size,xpix,ypix):
    '''
    Augment the data (3D array of images) by flipping and rotating the images.
    Size = upper bound on the final number of images 
    (actual_size can be much less depending on size/data_size multiples)
    
    TODO: Make the actual size = size
    '''
    rotations = size//len(data) # rotations per image
    angles = np.linspace(0, 360, rotations)
    act_size = rotations*len(data)
    training_set = np.empty((act_size, xpix, ypix))
    for i in range(len(data)):
        for j in range(len(angles)):
            if j % 2 == 0: training_set[i*len(angles)+j,:,:] = rotate(np.fliplr(data[i,:,:]), angles[j], reshape=False)
            else: training_set[i*len(angles)+j,:,:] = rotate(data[i,:,:], angles[j], reshape=False)
    return training_set

def train_test(data,percentage):
    '''
    Combines data sets in one 3D array, with a different label for each data set.
    Then randomly shuffles the data and splits into training and test sets.
    data = list 3D arrays containing desired data sets
    per = fraction of data to be in training set
    returns: train and test data (each a tupple containing the data and corresponding labels)
    '''
    d = np.concatenate(data,axis=0)
    n_images = len(d)
    labels = np.empty(n_images)
    i = 0
    for n in range(len(data)):
        labels[i:i+len(data[n])] = n
        i += len(data[n])
    rand_ind = np.random.permutation(range(n_images))
    d, labels = d[rand_ind], labels[rand_ind]
    n_train = np.int(np.round(n_images*percentage))
    train = (d[:n_train], labels[:n_train])
    test = (d[n_train:], labels[n_train:])
    return train, test

<font size = 3> Before the training set can be used, we have to split up the data according to bending angle and create a label for each image that corresponds to the category it has been placed in:

In [4]:
# Number of x pixels and y pixels in each image (must be the same for all images)
xpix, ypix = 83, 83

# Directories with the FITS data
agn_path = r'C:\Users\Cerys\Documents\Physics\Y4 Project\Data Preparation\Radio Zoo Images'
agn_data = get_clipped_images(agn_path,xpix,ypix,sigma=3)

In [5]:
datapath = r'C:\Users\Cerys\Documents\Physics\Y4 Project\Data Preparation\ra dec ang.txt'
agn_ang = np.array(pd.read_csv(datapath, sep='\s+', header=None, usecols=[2])) # bending angle of agn, from vizier catalog

In [6]:
# Define angle ranges for three different categories of AGN
ang1 = 20
ang2 = 45

# Assign each image in agn_data a label depending on which angle range it is in
labels = np.empty(len(agn_data))
for i in range(len(agn_ang)):
    if agn_ang[i] < ang1: labels[i] = 1
    elif agn_ang[i] > ang2: labels[i] = 3
    else: labels[i] = 2

In [7]:
cat1 = agn_data[labels==1]
cat2 = agn_data[labels==2]
cat3 = agn_data[labels==3]

In [27]:
# Augment the data
data = [] # empty list
for cat in (cat1,cat2,cat3):
    data.append(augment_data(cat,30000,xpix,ypix))

# Get training and test data and labels
train, test = train_test(data, 0.8)

## Aniyan Paper Model

<font size = 3> It was decided that the best approach to start this problem was to try and recreate the neural network seen in Aniyan's paper. The model below is as close a replica as possible to that in the paper and unfortunately the vast number of trainable parameters (58 million) made it too complicated to run on a regular machine. In order to test this network efficiently, we will need to use GPUs. 

In [15]:
# Aniyan Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(96, (11, 11), activation='relu', input_shape=(83, 83, 1)),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.MaxPooling2D((3, 3)),
    tf.keras.layers.Conv2D(256, (5, 5), activation='relu'),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(384, (3, 3), activation='relu'),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Conv2D(384, (3, 3), activation='relu'),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096*2, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax') ])
    
model.summary()

# Compile the network
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 73, 73, 96)        11712     
_________________________________________________________________
layer_normalization_4 (Layer (None, 73, 73, 96)        192       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 24, 24, 96)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 20, 20, 256)       614656    
_________________________________________________________________
layer_normalization_5 (Layer (None, 20, 20, 256)       512       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 10, 10, 256)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 8, 8, 384)        

In [ ]:
training = tf.reshape(train[0], [-1, 83, 83, 1])
labels = train[1]

model.fit(training, labels,epochs=5,verbose=1)

# Optimal Neural Network

<font size = 3> Due to the large runtime of the Aniyan model, it was better to start with something simpler (less trainable parameters) and see if that would work first. 

The options for refining the network included:

- Changing the number of convolutional, pooling, and densely connected layers.
- Changing the number of nodes at each layer.
- Chaniging the kernel size of the convolutional layers.
- Changing the activation function.

After trialling several attempts changing all of these parameters, the network below was decided upon to be the best. It only has 1 million trainable parameters resulting in a run time of around 5 hours, but the accuracy is around 97% which is much better than the simple neural net was for this problem (around 90%). 

In [7]:
# Augment the data
# Note - augmenting the data to around 40000 images produced the best possible results without overly increasing runtime.
# Would be worth checking if augmenting the data even further increases the accuracy, but not the priorty for this test.
for cat in (cat1,cat2,cat3):
    data.append(augment_data(cat,40000,xpix,ypix))

# Get training and test data and labels
train, test = train_test(data, 0.8)

In [8]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(83, 83, 1)))
model.add(tf.keras.layers.LayerNormalization())
model.add(tf.keras.layers.MaxPooling2D((3, 3)))
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(tf.keras.layers.LayerNormalization())
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.15)) 
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.summary()

# Compile the network
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 81, 81, 32)        320       
_________________________________________________________________
layer_normalization (LayerNo (None, 81, 81, 32)        64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 32)        9248      
_________________________________________________________________
layer_normalization_1 (Layer (None, 25, 25, 32)        64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

In [13]:
training = tf.reshape(train[0], [-1, 83, 83, 1])
labels = train[1]

model.fit(training, labels,epochs=20,verbose=1) 

Train on 94428 samples
Epoch 1/20
94428/94428 [==============================] - 731s 8ms/sample - loss: 0.0879 - accuracy: 0.9661
Epoch 2/20
94428/94428 [==============================] - 736s 8ms/sample - loss: 0.0927 - accuracy: 0.9658
Epoch 3/20
94428/94428 [==============================] - 749s 8ms/sample - loss: 0.0767 - accuracy: 0.9701
Epoch 4/20
94428/94428 [==============================] - 761s 8ms/sample - loss: 0.0780 - accuracy: 0.9700
Epoch 5/20
94428/94428 [==============================] - 1038s 11ms/sample - loss: 0.0757 - accuracy: 0.9707
Epoch 6/20
94428/94428 [==============================] - 1185s 13ms/sample - loss: 0.0809 - accuracy: 0.9694
Epoch 7/20
94428/94428 [==============================] - 1209s 13ms/sample - loss: 0.0661 - accuracy: 0.9739
Epoch 8/20
94428/94428 [==============================] - 1207s 13ms/sample - loss: 0.0700 - accuracy: 0.9731
Epoch 9/20
94428/94428 [==============================] - 1023s 11ms/sample - loss: 0.0675 - accuracy: 0.

In [ ]:
test = tf.reshape(train[0], [-1, 83, 83, 1])
test_labels = train[1]
test_loss, test_acc = model.evaluate(test,test_labels, verbose=1)
print('\nTest accuracy:', test_acc)